In [1]:
!pip install tensorflow-gpu==2.0.0beta1

In [2]:
!git clone https://github.com/ncoop57/gpt-2.git data/gpt-2

Cloning into 'data/gpt-2'...
remote: Enumerating objects: 319, done.
remote: Total 319 (delta 0), reused 0 (delta 0), pack-reused 319
Receiving objects: 100% (319/319), 4.40 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (177/177), done.


In [11]:
cd /tf/src/data/gpt-2/

/tf/src/data/gpt-2


In [5]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 604kB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 10.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 15.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 143kB 11.1MB/s eta 0:00:01
     |████████████████████████████████| 122kB 10.5MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex


In [6]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 985kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 9.56Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 714kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:43, 11.3Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 2.83Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 7.97Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 7.79Mit/s]                                                       


In [ ]:
!mkdir data
!wget -O data/training.txt https://raw.githubusercontent.com/micheletufano/NeuralCodeTranslator/master/dataset/bug-fixes/medium/train/fixed.txt # http://groups.inf.ed.ac.uk/cup/javaGithub/java_projects.tar.gz
!ls

In [ ]:
# cd src/
!ls src

In [ ]:
!tf_upgrade_v2 --infile src/encoder.py --outfile src/encoder-upgraded.py

In [ ]:
!cat src/encoder-upgraded.py

In [ ]:
!tf_upgrade_v2 --infile src/sample.py --outfile src/sample-upgraded.py

In [ ]:
!cat src/sample-upgraded.py

In [ ]:
!tf_upgrade_v2 --infile src/interactive_conditional_samples.py --outfile src/interactive_conditional_samples-upgraded.py

In [ ]:
!cat src/interactive_conditional_samples-upgraded.py

In [ ]:
!wget https://raw.githubusercontent.com/openai/gpt-2/master/src/model.py

In [ ]:
!tf_upgrade_v2 --infile model.py --outfile model-upgraded.py

In [ ]:
!cat model-upgraded.py

In [ ]:
!tf_upgrade_v2 --infile src/load_dataset.py --outfile src/load_dataset-upgraded.py

In [ ]:
!cat src/load_dataset-upgraded.py

In [ ]:
!tf_upgrade_v2 --infile src/memory_saving_gradients.py --outfile src/memory_saving_gradients-upgraded.py

In [ ]:
!cat src/memory_saving_gradients-upgraded.py

In [ ]:
!tf_upgrade_v2 --infile train.py --outfile train-upgraded.py

In [ ]:
!cat train-upgraded.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir data
! cp -r /content/drive/My\ Drive/data/ /content/data

In [ ]:
! unzip -qq /content/data//bug-fixes-methods.zip &

In [ ]:
! cp -r sciclone /content/drive/My\ Drive/data/

In [5]:
import fire
import json
import os
import numpy as np
import tensorflow as tf
import regex as re
from functools import lru_cache
import argparse
import time
import tqdm
from tensorflow.core.protobuf import rewriter_config_pb2
import glob

# from accumulate import AccumulatingOptimizer
# import memory_saving_gradients

tf.__version__

'2.0.0-beta1'

In [4]:
! pip install -r /tf/src/data/gpt-2/requirements.txt

     |████████████████████████████████| 81kB 3.6MB/s eta 0:00:011
     |████████████████████████████████| 604kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 15.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 13.7MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: urllib3 1.25.3
    Uninstalling urllib3-1.25.3:
      Successfully uninstalled urllib3-1.25.3
  Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
  Found existing installation: tqdm 4.34.0
    Uninstalling tqdm-4.34.0:
      Successfully uninstalled tqdm-4.34.0
You should consider upgrading via the 'pip install --up

In [ ]:
import tensorflow as tf

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)
# def create_look_ahead_mask(size):
#   mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
#   return mask

x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp * -1e9

# Encoding

In [9]:
"""Byte pair encoding utilities"""


@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

def get_pairs(word):
    """Return set of symbol pairs in a word.

    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

class Encoder:
    def __init__(self, encoder, bpe_merges, errors='replace'):
        self.encoder = encoder
        self.decoder = {v:k for k,v in self.encoder.items()}
        self.errors = errors # how to handle errors in decoding
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v:k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}

        # Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors=self.errors)
        return text

def get_encoder(model_name, models_dir):
    with open(os.path.join(models_dir, model_name, 'encoder.json'), 'r') as f:
        encoder = json.load(f)
    with open(os.path.join(models_dir, model_name, 'vocab.bpe'), 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    return Encoder(
        encoder=encoder,
        bpe_merges=bpe_merges,
    )

# Model

In [13]:
class HParams():
  n_vocab=50257
  n_ctx=1024
  n_embd=768
  n_head=12
  n_layer=12
  
  def __init__(self, n_vocab, n_ctx, n_embd, n_head, n_layer):
    self.n_vocab = n_vocab
    self.n_ctx = n_ctx
    self.n_embd = n_embd
    self.n_head = n_head
    self.n_layer = n_layer

In [6]:
def default_hparams():
    return HParams(
        n_vocab=50257,
        n_ctx=1024,
        n_embd=768,
        n_head=12,
        n_layer=12,
    )

def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(input=x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

def norm(x, scope, *, axis=-1, epsilon=1e-5):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    with tf.compat.v1.variable_scope(scope):
        n_state = x.shape[-1]
        g = tf.compat.v1.get_variable('g', [n_state], initializer=tf.compat.v1.constant_initializer(1), use_resource=False)
        b = tf.compat.v1.get_variable('b', [n_state], initializer=tf.compat.v1.constant_initializer(0), use_resource=False)
        u = tf.reduce_mean(input_tensor=x, axis=axis, keepdims=True)
        s = tf.reduce_mean(input_tensor=tf.square(x-u), axis=axis, keepdims=True)
        x = (x - u) * tf.math.rsqrt(s + epsilon)
        x = x*g + b
        return x

def split_states(x, n):
    """Reshape the last dimension of x into [n, x.shape[-1]/n]."""
    *start, m = shape_list(x)
    return tf.reshape(x, start + [n, m//n])

def merge_states(x):
    """Smash the last two dimensions of x into a single dimension."""
    *start, a, b = shape_list(x)
    return tf.reshape(x, start + [a*b])

def conv1d(x, scope, nf, *, w_init_stdev=0.02):
    with tf.compat.v1.variable_scope(scope):
        *start, nx = shape_list(x)
        w = tf.compat.v1.get_variable('w', [1, nx, nf], initializer=tf.compat.v1.random_normal_initializer(stddev=w_init_stdev), use_resource=False)
        b = tf.compat.v1.get_variable('b', [nf], initializer=tf.compat.v1.constant_initializer(0), use_resource=False)
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

def attention_mask(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.

    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= j - ns + nd
    return tf.cast(m, dtype)


def attn(x, scope, n_state, *, past, hparams):
    assert x.shape.ndims == 3  # Should be [batch, sequence, features]
    assert n_state % hparams.n_head == 0
    if past is not None:
        assert past.shape.ndims == 5  # Should be [batch, 2, heads, sequence, features], where 2 is [k, v]

    def split_heads(x):
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(a=split_states(x, hparams.n_head), perm=[0, 2, 1, 3])

    def merge_heads(x):
        # Reverse of split_heads
        return merge_states(tf.transpose(a=x, perm=[0, 2, 1, 3]))

    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        _, _, nd, ns = shape_list(w)
        b = attention_mask(nd, ns, dtype=w.dtype)
        b = tf.reshape(b, [1, 1, nd, ns])
        w = w*b - tf.cast(1e10, w.dtype)*(1-b)
        return w

    def multihead_attn(q, k, v):
        # q, k, v have shape [batch, heads, sequence, features]
        w = tf.matmul(q, k, transpose_b=True)
        w = w * tf.math.rsqrt(tf.cast(v.shape[-1], w.dtype))

        w = mask_attn_weights(w)
        w = tf.nn.softmax(w, axis=-1)
#         w = softmax(w)
        a = tf.matmul(w, v)
        return a

    with tf.compat.v1.variable_scope(scope):
        c = conv1d(x, 'c_attn', n_state*3)
        q, k, v = map(split_heads, tf.split(c, 3, axis=2))
        present = tf.stack([k, v], axis=1)
        if past is not None:
            pk, pv = tf.unstack(past, axis=1)
            k = tf.concat([pk, k], axis=-2)
            v = tf.concat([pv, v], axis=-2)
        a = multihead_attn(q, k, v)
        a = merge_heads(a)
        a = conv1d(a, 'c_proj', n_state)
        return a, present


def mlp(x, scope, n_state, *, hparams):
    with tf.compat.v1.variable_scope(scope):
        nx = x.shape[-1]
        h = gelu(conv1d(x, 'c_fc', n_state))
        h2 = conv1d(h, 'c_proj', nx)
        return h2

def block(x, scope, *, past, hparams):
    with tf.compat.v1.variable_scope(scope):
        nx = x.shape[-1]
#         layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5, name='ln_1')
#         a, present = attn(layer_norm1(x), 'attn', nx, past=past, hparams=hparams)
        a, present = attn(norm(x, 'ln_1'), 'attn', nx, past=past, hparams=hparams)
        x = x + a
        m = mlp(norm(x, 'ln_2'), 'mlp', nx*4, hparams=hparams)
        x = x + m
        return x, present

def past_shape(*, hparams, batch_size=None, sequence=None):
    return [batch_size, hparams.n_layer, 2, hparams.n_head, sequence, hparams.n_embd // hparams.n_head]

def expand_tile(value, size):
    """Add a new axis of given size."""
    value = tf.convert_to_tensor(value=value, name='value')
    ndims = value.shape.ndims
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)

def positions_for(tokens, past_length):
    batch_size = tf.shape(input=tokens)[0]
    nsteps = tf.shape(input=tokens)[1]
    return expand_tile(past_length + tf.range(nsteps), batch_size)


def model(hparams, X, past=None, scope='model', reuse=tf.compat.v1.AUTO_REUSE):
    with tf.compat.v1.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wpe = tf.compat.v1.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
                             initializer=tf.compat.v1.random_normal_initializer(stddev=0.01), use_resource=False)
        wte = tf.compat.v1.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.compat.v1.random_normal_initializer(stddev=0.02), use_resource=False)
        past_length = 0 if past is None else tf.shape(input=past)[-2]
        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))

        # Transformer
        presents = []
        pasts = tf.unstack(past, axis=1) if past is not None else [None] * hparams.n_layer
        assert len(pasts) == hparams.n_layer
        for layer, past in enumerate(pasts):
            h, present = block(h, 'h%d' % layer, past=past, hparams=hparams)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits
        return results

# Sample from Model

In [7]:
def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.compat.v1.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    return tf.cond(
       pred=tf.equal(k, 0),
       true_fn=lambda: logits,
       false_fn=lambda: _top_k(),
    )


def sample_sequence(*, hparams, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = tf.fill([batch_size, 1], start_token)

    def step(hparams, tokens, past=None):
        lm_output = model(hparams=hparams, X=tokens, past=past, reuse=tf.compat.v1.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(past_shape(hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents,
        }

#     with tf.compat.v1.name_scope('sample_sequence'):
    def body(past, prev, output):
        next_outputs = step(hparams, prev, past=past)
        logits = next_outputs['logits'][:, -1, :]  / tf.cast(temperature, dtype=tf.float32)
        logits = top_k_logits(logits, k=top_k)
        samples = tf.random.categorical(logits=logits, num_samples=1, dtype=tf.int32)
        return [
            next_outputs['presents'] if past is None else tf.concat([past, next_outputs['presents']], axis=-2),
            samples,
            tf.concat([output, samples], axis=1)
        ]

    past, prev, output = body(None, context, context)

    def cond(*args):
        return True
#     print(length)
#         for _ in range(20):
#             past, prev, output = body(past, prev, output)

    _, _, tokens = tf.while_loop(
        cond=cond, body=body,
        maximum_iterations=length - 1,
        loop_vars=[
            past,
            prev,
            output
        ],
        shape_invariants=[
            tf.TensorShape(past_shape(hparams=hparams, batch_size=batch_size)),
            tf.TensorShape([batch_size, None]),
            tf.TensorShape([batch_size, None]),
        ],
        back_prop=False,
    )

    return tokens

In [ ]:
!ls models/345M
!cat models/117M/hparams.json

# Interactive Mode

In [ ]:
def interact_model(
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',    
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)     
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = get_encoder(model_name, models_dir)
    hparams = default_hparams()
#     with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
#         hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        context = tf.compat.v1.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.compat.v1.set_random_seed(seed)
        output = sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )
        

        saver = tf.compat.v1.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)
        tf.compat.v1.global_variables_initializer()
#         init = tf.compat.v1.global_variables_initializer()
#         sess.run(init)
        

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                print("Output Obj: ", output)
                print("Context: ", [context_tokens for _ in range(batch_size)])
#                 out = output #predict(output, length, [context_tokens for _ in range(batch_size)])
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

interact_model(model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=40,
    models_dir='models')

# if __name__ == '__main__':
#     fire.Fire(interact_model)

W0904 20:19:02.771937 140475408045888 deprecation.py:323] From <ipython-input-7-ef3b52cbd52e>:12: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0904 20:19:05.333244 140475408045888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Model prompt >>> Hi, my name is
Output Obj:  Tensor("while/Exit_3:0", shape=(1, None), dtype=int32)
Context:  [[17250, 11, 616, 1438, 318]]
======================================== SAMPLE 1 ========================================
 Ryan Kostka. I've gone to Harvard, the most prestigious university in America; you can't get there in a black shirt. If you do find something of value at all, make us a call. Call me. Please.

That's it. I'll be here. Good to see you, there. Thank you for leaving.<|endoftext|>WASHINGTON, July 5 (UPI) -- President Obama gave a televised address on Friday thanking Russia for helping him win a military victory in Ukraine, but also acknowledging that more than 600 troops still have not returned to the country, and that many in the U.S. believe the United States can't afford a second war, in which U.S.-backed rebels have killed thousands.

The President addressed the United States' military personnel at the National Defense Reserves Training Center, located at th

In [ ]:
ates(x):
    """Smash the last twfrom toposort import toposort
import contextlib
import numpy as np
import tensorflow as tf
import tensorflow.contrib.graph_editor as ge
import time
import sys
sys.setrecursionlimit(10000)
# refers back to current module if we decide to split helpers out
util = sys.modules[__name__]

# getting rid of "WARNING:tensorflow:VARIABLES collection name is deprecated"
setattr(tf.compat.v1.GraphKeys, "VARIABLES", "variables")

# save original gradients since tf.gradient could be monkey-patched to point
# to our version
from tensorflow.python.ops import gradients as tf_gradients_lib
tf_gradients = tf_gradients_lib.gradients

MIN_CHECKPOINT_NODE_SIZE=1024    # use lower value during testing

# specific versions we can use to do process-wide replacement of tf.gradients
def gradients_speed(ys, xs, grad_ys=None, **kwargs):
    return gradients(ys, xs, grad_ys, checkpoints='speed', **kwargs)

def gradients_memory(ys, xs, grad_ys=None, **kwargs):
    return gradients(ys, xs, grad_ys, checkpoints='memory', **kwargs)

def gradients_collection(ys, xs, grad_ys=None, **kwargs):
    return gradients(ys, xs, grad_ys, checkpoints='collection', **kwargs)

def gradients(ys, xs, grad_ys=None, checkpoints='collection', **kwargs):
    '''
    Authors: Tim Salimans & Yaroslav Bulatov

    memory efficient gradient implementation inspired by "Training Deep Nets with Sublinear Memory Cost"
    by Chen et al. 2016 (https://arxiv.org/abs/1604.06174)

    ys,xs,grad_ys,kwargs are the arguments to standard tensorflow tf.gradients
    (https://www.tensorflow.org/versions/r0.12/api_docs/python/train.html#gradients)

    'checkpoints' can either be
        - a list consisting of tensors from the forward pass of the neural net
          that we should re-use when calculating the gradients in the backward pass
          all other tensors that do not appear in this list will be re-computed
        - a string specifying how this list should be determined. currently we support
            - 'speed':  checkpoint all outputs of convolutions and matmuls. these ops are usually the most expensive,
                        so checkpointing them maximizes the running speed
                        (this is a good option if nonlinearities, concats, batchnorms, etc are taking up a lot of memory)
            - 'memory': try to minimize the memory usage
                        (currently using a very simple strategy that identifies a number of bottleneck tensors in the graph to checkpoint)
            - 'collection': look for a tensorflow collection named 'checkpoints', which holds the tensors to checkpoint
    '''

    #    print("Calling memsaving gradients with", checkpoints)
    if not isinstance(ys,list):
        ys = [ys]
    if not isinstance(xs,list):
        xs = [xs]

    bwd_ops = ge.get_backward_walk_ops([y.op for y in ys],
                                       inclusive=True)

    debug_print("bwd_ops: %s", bwd_ops)

    # forward ops are all ops that are candidates for recomputation
    fwd_ops = ge.get_forward_walk_ops([x.op for x in xs],
                                      inclusive=True,
                                      within_ops=bwd_ops)
    debug_print("fwd_ops: %s", fwd_ops)

    # exclude ops with no inputs
    fwd_ops = [op for op in fwd_ops if op.inputs]

    # don't recompute xs, remove variables
    xs_ops = _to_ops(xs)
    fwd_ops = [op for op in fwd_ops if not op in xs_ops]
    fwd_ops = [op for op in fwd_ops if not '/assign' in op.name]
    fwd_ops = [op for op in fwd_ops if not '/Assign' in op.name]
    fwd_ops = [op for op in fwd_ops if not '/read' in op.name]
    ts_all = ge.filter_ts(fwd_ops, True) # get the tensors
    ts_all = [t for t in ts_all if '/read' not in t.name]
    ts_all = set(ts_all) - set(xs) - set(ys)

    # construct list of tensors to checkpoint during forward pass, if not
    # given as input
    if type(checkpoints) is not list:
        if checkpoints == 'collection':
            checkpoints = tf.compat.v1.get_collection('checkpoints')

        elif checkpoints == 'speed':
            # checkpoint all expensive ops to maximize running speed
            checkpoints = ge.filter_ts_from_regex(fwd_ops, 'conv2d|Conv|MatMul')

        elif checkpoints == 'memory':

            # remove very small tensors and some weird ops
            def fixdims(t): # tf.Dimension values are not compatible with int, convert manually
                try:
                    return [int(e if e.value is not None else 64) for e in t]
                except:
                    return [0]  # unknown shape
            ts_all = [t for t in ts_all if np.prod(fixdims(t.shape)) > MIN_CHECKPOINT_NODE_SIZE]
            ts_all = [t for t in ts_all if 'L2Loss' not in t.name]
            ts_all = [t for t in ts_all if 'entropy' not in t.name]
            ts_all = [t for t in ts_all if 'FusedBatchNorm' not in t.name]
            ts_all = [t for t in ts_all if 'Switch' not in t.name]
            ts_all = [t for t in ts_all if 'dropout' not in t.name]
            # DV: FP16_FIX - need to add 'Cast' layer here to make it work for FP16
            ts_all = [t for t in ts_all if 'Cast' not in t.name]

            # filter out all tensors that are inputs of the backward graph
            with util.capture_ops() as bwd_ops:
                tf_gradients(ys, xs, grad_ys, **kwargs)

            bwd_inputs = [t for op in bwd_ops for t in op.inputs]
            # list of tensors in forward graph that is in input to bwd graph
            ts_filtered = list(set(bwd_inputs).intersection(ts_all))
            debug_print("Using tensors %s", ts_filtered)

            # try two slightly different ways of getting bottlenecks tensors
            # to checkpoint
            for ts in [ts_filtered, ts_all]:

                # get all bottlenecks in the graph
                bottleneck_ts = []
                for t in ts:
                    b = set(ge.get_backward_walk_ops(t.op, inclusive=True, within_ops=fwd_ops))
                    f = set(ge.get_forward_walk_ops(t.op, inclusive=False, within_ops=fwd_ops))
                    # check that there are not shortcuts
                    b_inp = set([inp for op in b for inp in op.inputs]).intersection(ts_all)
                    f_inp = set([inp for op in f for inp in op.inputs]).intersection(ts_all)
                    if not set(b_inp).intersection(f_inp) and len(b_inp)+len(f_inp) >= len(ts_all):
                        bottleneck_ts.append(t)  # we have a bottleneck!
                    else:
                        debug_print("Rejected bottleneck candidate and ops %s", [t] + list(set(ts_all) - set(b_inp) - set(f_inp)))

                # success? or try again without filtering?
                if len(bottleneck_ts) >= np.sqrt(len(ts_filtered)): # yes, enough bottlenecks found!
                    break

            if not bottleneck_ts:
                raise Exception('unable to find bottleneck tensors! please provide checkpoint nodes manually, or use checkpoints="speed".')

            # sort the bottlenecks
            bottlenecks_sorted_lists = tf_toposort(bottleneck_ts, within_ops=fwd_ops)
            sorted_bottlenecks = [t for ts in bottlenecks_sorted_lists for t in ts]

            # save an approximately optimal number ~ sqrt(N)
            N = len(ts_filtered)
            if len(bottleneck_ts) <= np.ceil(np.sqrt(N)):
                checkpoints = sorted_bottlenecks
            else:
                step = int(np.ceil(len(bottleneck_ts) / np.sqrt(N)))
                checkpoints = sorted_bottlenecks[step::step]

        else:
            raise Exception('%s is unsupported input for "checkpoints"' % (checkpoints,))

    checkpoints = list(set(checkpoints).intersection(ts_all))

    # at this point automatic selection happened and checkpoints is list of nodes
    assert isinstance(checkpoints, list)

    debug_print("Checkpoint nodes used: %s", checkpoints)
    # better error handling of special cases
    # xs are already handled as checkpoint nodes, so no need to include them
    xs_intersect_checkpoints = set(xs).intersection(set(checkpoints))
    if xs_intersect_checkpoints:
        debug_print("Warning, some input nodes are also checkpoint nodes: %s",
                    xs_intersect_checkpoints)
    ys_intersect_checkpoints = set(ys).intersection(set(checkpoints))
    debug_print("ys: %s, checkpoints: %s, intersect: %s", ys, checkpoints,
                ys_intersect_checkpoints)
    # saving an output node (ys) gives no benefit in memory while creating
    # new edge cases, exclude them
    if ys_intersect_checkpoints:
        debug_print("Warning, some output nodes are also checkpoints nodes: %s",
              format_ops(ys_intersect_checkpoints))

    # remove initial and terminal nodes from checkpoints list if present
    checkpoints = list(set(checkpoints) - set(ys) - set(xs))

    # check that we have some nodes to checkpoint
    # if not checkpoints:
    #     raise Exception('no checkpoints nodes found or given as input! ')

    # disconnect dependencies between checkpointed tensors
    checkpoints_disconnected = {}
    for x in checkpoints:
        if x.op and x.op.name is not None:
            grad_node = tf.stop_gradient(x, name=x.op.name+"_sg")
        else:
            grad_node = tf.stop_gradient(x)
        checkpoints_disconnected[x] = grad_node

    # partial derivatives to the checkpointed tensors and xs
    ops_to_copy = fast_backward_ops(seed_ops=[y.op for y in ys],
                                    stop_at_ts=checkpoints, within_ops=fwd_ops)
    debug_print("Found %s ops to copy within fwd_ops %s, seed %s, stop_at %s",
                    len(ops_to_copy), fwd_ops, [r.op for r in ys], checkpoints)
    debug_print("ops_to_copy = %s", ops_to_copy)
    debug_print("Processing list %s", ys)
    copied_sgv, info = ge.copy_with_input_replacements(ge.sgv(ops_to_copy), {})
    for origin_op, op in info._transformed_ops.items():
        op._set_device(origin_op.node_def.device)
    copied_ops = info._transformed_ops.values()
    debug_print("Copied %s to %s", ops_to_copy, copied_ops)
    ge.reroute_ts(checkpoints_disconnected.values(), checkpoints_disconnected.keys(), can_modify=copied_ops)
    debug_print("Rewired %s in place of %s restricted to %s",
                checkpoints_disconnected.values(), checkpoints_disconnected.keys(), copied_ops)

    # get gradients with respect to current boundary + original x's
    copied_ys = [info._transformed_ops[y.op]._outputs[0] for y in ys]
    boundary = list(checkpoints_disconnected.values())
    dv = tf_gradients(ys=copied_ys, xs=boundary+xs, grad_ys=grad_ys, **kwargs)
    debug_print("Got gradients %s", dv)
    debug_print("for %s", copied_ys)
    debug_print("with respect to %s", boundary+xs)

    inputs_to_do_before = [y.op for y in ys]
    if grad_ys is not None:
        inputs_to_do_before += grad_ys
    wait_to_do_ops = list(copied_ops) + [g.op for g in dv if g is not None]
    my_add_control_inputs(wait_to_do_ops, inputs_to_do_before)

    # partial derivatives to the checkpointed nodes
    # dictionary of "node: backprop" for nodes in the boundary
    d_checkpoints = {r: dr for r,dr in zip(checkpoints_disconnected.keys(),
                                        dv[:len(checkpoints_disconnected)])}
    # partial derivatives to xs (usually the params of the neural net)
    d_xs = dv[len(checkpoints_disconnected):]

    # incorporate derivatives flowing through the checkpointed nodes
    checkpoints_sorted_lists = tf_toposort(checkpoints, within_ops=fwd_ops)
    for ts in checkpoints_sorted_lists[::-1]:
        debug_print("Processing list %s", ts)
        checkpoints_other = [r for r in checkpoints if r not in ts]
        checkpoints_disconnected_other = [checkpoints_disconnected[r] for r in checkpoints_other]

        # copy part of the graph below current checkpoint node, stopping at
        # other checkpoints nodes
        ops_to_copy = fast_backward_ops(within_ops=fwd_ops, seed_ops=[r.op for r in ts], stop_at_ts=checkpoints_other)
        debug_print("Found %s ops to copy within %s, seed %s, stop_at %s",
                    len(ops_to_copy), fwd_ops, [r.op for r in ts],
                    checkpoints_other)
        debug_print("ops_to_copy = %s", ops_to_copy)
        if not ops_to_copy: # we're done!
            break
        copied_sgv, info = ge.copy_with_input_replacements(ge.sgv(ops_to_copy), {})
        for origin_op, op in info._transformed_ops.items():
            op._set_device(origin_op.node_def.device)
        copied_ops = info._transformed_ops.values()
        debug_print("Copied %s to %s", ops_to_copy, copied_ops)
        ge.reroute_ts(checkpoints_disconnected_other, checkpoints_other, can_modify=copied_ops)
        debug_print("Rewired %s in place of %s restricted to %s",
                    checkpoints_disconnected_other, checkpoints_other, copied_ops)

        # gradient flowing through the checkpointed node
        boundary = [info._transformed_ops[r.op]._outputs[0] for r in ts]
        substitute_backprops = [d_checkpoints[r] for r in ts]
        dv = tf_gradients(boundary,
                          checkpoints_disconnected_other+xs,
                          grad_ys=substitute_backprops, **kwargs)
        debug_print("Got gradients %s", dv)
        debug_print("for %s", boundary)
        debug_print("with respect to %s", checkpoints_disconnected_other+xs)
        debug_print("with boundary backprop substitutions %s", substitute_backprops)

        inputs_to_do_before = [d_checkpoints[r].op for r in ts]
        wait_to_do_ops = list(copied_ops) + [g.op for g in dv if g is not None]
        my_add_control_inputs(wait_to_do_ops, inputs_to_do_before)

        # partial derivatives to the checkpointed nodes
        for r, dr in zip(checkpoints_other, dv[:len(checkpoints_other)]):
            if dr is not None:
                if d_checkpoints[r] is None:
                    d_checkpoints[r] = dr
                else:
                    d_checkpoints[r] += dr
        def _unsparsify(x):
            if not isinstance(x, tf.IndexedSlices):
                return x
            assert x.dense_shape is not None, "memory_saving_gradients encountered sparse gradients of unknown shape"
            indices = x.indices
            while indices.shape.ndims < x.values.shape.ndims:
                indices = tf.expand_dims(indices, -1)
            return tf.scatter_nd(indices, x.values, x.dense_shape)

        # partial derivatives to xs (usually the params of the neural net)
        d_xs_new = dv[len(checkpoints_other):]
        for j in range(len(xs)):
            if d_xs_new[j] is not None:
                if d_xs[j] is None:
                    d_xs[j] = _unsparsify(d_xs_new[j])
                else:
                    d_xs[j] += _unsparsify(d_xs_new[j])


    return d_xs

def tf_toposort(ts, within_ops=None):
    all_ops = ge.get_forward_walk_ops([x.op for x in ts], within_ops=within_ops)

    deps = {}
    for op in all_ops:
        for o in op.outputs:
            deps[o] = set(op.inputs)
    sorted_ts = toposort(deps)

    # only keep the tensors from our original list
    ts_sorted_lists = []
    for l in sorted_ts:
        keep = list(set(l).intersection(ts))
        if keep:
            ts_sorted_lists.append(keep)

    return ts_sorted_lists

def fast_backward_ops(within_ops, seed_ops, stop_at_ts):
    bwd_ops = set(ge.get_backward_walk_ops(seed_ops, stop_at_ts=stop_at_ts))
    ops = bwd_ops.intersection(within_ops).difference([t.op for t in stop_at_ts])
    return list(ops)

@contextlib.contextmanager
def capture_ops():
  """Decorator to capture ops created in the block.
  with capture_ops() as ops:
    # create some ops
  print(ops) # => prints ops created.
  """

  micros = int(time.time()*10**6)
  scope_name = str(micros)
  op_list = []
  with tf.compat.v1.name_scope(scope_name):
    yield op_list

  g = tf.compat.v1.get_default_graph()
  op_list.extend(ge.select_ops(scope_name+"/.*", graph=g))

def _to_op(tensor_or_op):
  if hasattr(tensor_or_op, "op"):
    return tensor_or_op.op
  return tensor_or_op

def _to_ops(iterable):
  if not _is_iterable(iterable):
    return iterable
  return [_to_op(i) for i in iterable]

def _is_iterable(o):
  try:
    _ = iter(o)
  except Exception:
    return False
  return True

DEBUG_LOGGING=False
def debug_print(s, *args):
  """Like logger.log, but also replaces all TensorFlow ops/tensors with their
  names. Sensitive to value of DEBUG_LOGGING, see enable_debug/disable_debug

  Usage:
    debug_print("see tensors %s for %s", tensorlist, [1,2,3])
  """

  if DEBUG_LOGGING:
    formatted_args = [format_ops(arg) for arg in args]
    print("DEBUG "+s % tuple(formatted_args))

def format_ops(ops, sort_outputs=True):
  """Helper method for printing ops. Converts Tensor/Operation op to op.name,
  rest to str(op)."""

  if hasattr(ops, '__iter__') and not isinstance(ops, str):
    l = [(op.name if hasattr(op, "name") else str(op)) for op in ops]
    if sort_outputs:
      return sorted(l)
    return l
  else:
    return ops.name if hasattr(ops, "name") else str(ops)

def my_add_control_inputs(wait_to_do_ops, inputs_to_do_before):
    for op in wait_to_do_ops:
        ci = [i for i in inputs_to_do_before if op.control_inputs is None or i not in op.control_inputs]
        ge.add_control_inputs(op, ci)

In [12]:
from pathlib import Path
def load_dataset(enc, path, combine):
    paths = []
#     paths = Path(path).glob('**/*.java')
#       print(i)
# #     if os.path.isfile(path):
# #         # Simple file
# #         paths.append(path)
#     elif os.path.isdir(path):
        # Directory
#     for (dirpath, _, fnames) in os.walk(path):
#         for fname in fnames:
#             paths.append(os.path.join(dirpath, fname))
# #     else:
# #         # Assume glob
#     paths = glob.glob(path, recursive=True)
    if os.path.isfile(path):
        # Simple file
        paths.append(path)
    elif os.path.isdir(path):
        # Directory
        for i, (dirpath, _, fnames) in enumerate(os.walk(path)):
            if i % 5000 == 0:
              print(i)
            for fname in fnames:
                paths.append(os.path.join(dirpath, fname))
                
            if i == 50000:
              print("Breaking")
              break
    else:
        # Assume glob
        paths = glob.glob(path)

        
#     print("Paths:", paths)
    token_chunks = []
#     new_paths = list()
    raw_text = ''
    for i, path in enumerate(tqdm.tqdm(paths)):
#         print("Path:", path)
        if 'after.java' not in path:
          continue
#         if path.endswith('.npz'):
#             # Pre-encoded
#             with np.load(path) as npz:
#                 for item in npz.files:
#                     token_chunks.append(npz[item])
#         else:
          # Plain text

        try:
            with open(path, 'r') as fp:
                raw_text += fp.read()
            tokens = np.stack(enc.encode(raw_text))
            token_chunks.append(tokens)
            raw_text = ''
#                 methods = raw_text.splitlines()
#                 for method in tqdm.tqdm(methods):
#                     tokens = np.stack(enc.encode(method))
#                     token_chunks.append(tokens)
        except:
            print(e)
            if i % 100 == 0:
              print("Methods:", len(token_chunks))
        if i >= 100000:
          break
#    if raw_text:
#        tokens = np.stack(enc.encode(raw_text))
#        token_chunks.append(tokens)
    return token_chunks

def wrapper(paths):
  while True:
    try:
      yield next(paths)
    except StopIteration:
      raise
    except Exception as e:
      print(e) # or whatever kind of logging you want
      pass

def binary_search(f, lo, hi):
    if f(lo) or not f(hi):
        return None
    while hi > lo + 1:
        mid = (lo + hi) // 2
        if f(mid):
            hi = mid
        else:
            lo = mid
    return hi


class Sampler(object):
    """Fairly samples a slice from a set of variable sized chunks.

    'Fairly' means that the distribution is the same as sampling from one concatenated chunk,
    but without crossing chunk boundaries."""

    def __init__(self, chunks, seed=None):
        self.chunks = chunks
        self.total_size = sum(chunk.shape[0] for chunk in chunks)
        self.boundaries = [0]
        for i in range(len(chunks)):
            self.boundaries.append(self.boundaries[-1] + chunks[i].shape[0])
        self.rs = np.random.RandomState(seed=seed)

    def sample(self, length):
        assert length < self.total_size // len(
            self.chunks
        ), "Dataset files are too small to sample {} tokens at a time".format(
            length)
        while True:
            index = self.rs.randint(0, self.total_size - length - 1)
            i = binary_search(lambda j: self.boundaries[j] > index, 0,
                              len(self.boundaries) - 1) - 1
            if self.boundaries[i + 1] > index + length:
                within_chunk = index - self.boundaries[i]
                return self.chunks[i][within_chunk:within_chunk + length]

In [13]:
class Args():
  def __init__(self, dataset, model_name, combine, batch_size, learning_rate, optimizer, noise, top_k, top_p, run_name, sample_every, sample_length, sample_num, save_every, val_dataset, val_batch_size, val_batch_count, val_every):
    self.dataset = dataset
    self.model_name = model_name
    self.combine = combine
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.optimizer = optimizer
    self.noise = noise
    self.top_k = top_k
    self.top_p = top_p
    self.run_name = run_name
    self.sample_every = sample_every
    self.sample_length = sample_length
    self.sample_num = sample_num
    self.save_every = save_every
    self.val_dataset = val_dataset
    self.val_batch_size = val_batch_size
    self.val_batch_count = val_batch_count
    self.val_every = val_every

In [17]:
args = Args(
                dataset="../sciclone/data10/mtufano/deepLearningMutants/out/changes/code",
                model_name="117M",
                combine=50000,
                batch_size=1,
                learning_rate=0.00002,
                optimizer="sgd",
                noise=0.0,
                top_k=40,
                top_p=0.0,
                run_name="run1",
                sample_every=100,
                sample_length=1023,
                sample_num=1,
                save_every=1000,
                val_dataset=None,
                val_batch_size=2,
                val_batch_count=40,
                val_every=0
    )
enc = get_encoder(args.model_name, "models")

data_set = load_dataset(enc, args.dataset, args.combine)
len(data_set)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000


  0%|          | 22/115158 [00:00<10:50, 176.88it/s]

50000
Breaking


 87%|████████▋ | 99944/115158 [03:59<00:26, 581.30it/s]

25358

 87%|████████▋ | 99944/115158 [04:10<00:26, 581.30it/s]

In [18]:
DATA_SET_SIZE = len(data_set)
TRN_SET_SIZE = int(DATA_SET_SIZE * 0.8)
VAL_SET_SIZE = int(DATA_SET_SIZE * 0.1)
TST_SET_SIZE = int(DATA_SET_SIZE * 0.1)

trn_set = data_set[:TRN_SET_SIZE]
val_set = data_set[TRN_SET_SIZE:TRN_SET_SIZE + VAL_SET_SIZE]
tst_set = data_set[-TST_SET_SIZE:]
len(trn_set), len(val_set), len(tst_set)
# print(len(trn_set), len(val_set), len(tst_set))

# print(len(trn_set) + len(val_set) + len(tst_set), DATA_SET_SIZE)

(20286, 2535, 2535)

In [ ]:
CHECKPOINT_DIR = 'checkpoint'
SAMPLE_DIR = 'samples'


# parser = argparse.ArgumentParser(
#     description='Fine-tune GPT-2 on your custom dataset.',
#     formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# parser.add_argument('--dataset', metavar='PATH', type=str, required=True, help='Input file, directory, or glob pattern (utf-8 text, or preencoded .npz files).')
# parser.add_argument('--model_name', metavar='MODEL', type=str, default='117M', help='Pretrained model name')
# parser.add_argument('--combine', metavar='CHARS', type=int, default=50000, help='Concatenate input files with <|endoftext|> separator into chunks of this minimum size')

# parser.add_argument('--batch_size', metavar='SIZE', type=int, default=1, help='Batch size')
# parser.add_argument('--learning_rate', metavar='LR', type=float, default=0.00002, help='Learning rate for Adam')
# parser.add_argument('--accumulate_gradients', metavar='N', type=int, default=1, help='Accumulate gradients across N minibatches.')
# parser.add_argument('--memory_saving_gradients', default=False, action='store_true', help='Use gradient checkpointing to reduce vram usage.')
# parser.add_argument('--only_train_transformer_layers', default=False, action='store_true', help='Restrict training to the transformer blocks.')
# parser.add_argument('--optimizer', type=str, default='adam', help='Optimizer. <adam|sgd>.')
# parser.add_argument('--noise', type=float, default=0.0, help='Add noise to input training data to regularize against typos.')

# parser.add_argument('--top_k', type=int, default=40, help='K for top-k sampling.')
# parser.add_argument('--top_p', type=float, default=0.0, help='P for top-p sampling. Overrides top_k if set > 0.')

# parser.add_argument('--restore_from', type=str, default='latest', help='Either "latest", "fresh", or a path to a checkpoint file')
# parser.add_argument('--run_name', type=str, default='run1', help='Run id. Name of subdirectory in checkpoint/ and samples/')
# parser.add_argument('--sample_every', metavar='N', type=int, default=100, help='Generate samples every N steps')
# parser.add_argument('--sample_length', metavar='TOKENS', type=int, default=1023, help='Sample this many tokens')
# parser.add_argument('--sample_num', metavar='N', type=int, default=1, help='Generate this many samples')
# parser.add_argument('--save_every', metavar='N', type=int, default=1000, help='Write a checkpoint every N steps')

# parser.add_argument('--val_dataset', metavar='PATH', type=str, default=None, help='Dataset for validation loss, defaults to --dataset.')
# parser.add_argument('--val_batch_size', metavar='SIZE', type=int, default=2, help='Batch size for validation.')
# parser.add_argument('--val_batch_count', metavar='N', type=int, default=40, help='Number of batches for validation.')
# parser.add_argument('--val_every', metavar='STEPS', type=int, default=0, help='Calculate validation loss every STEPS steps.')


def maketree(path):
    try:
        os.makedirs(path)
    except:
        pass


def randomize(context, hparams, p):
    if p > 0:
        mask = tf.random.uniform(shape=tf.shape(input=context)) < p
        noise = tf.random.uniform(shape=tf.shape(input=context), minval=0, maxval=hparams.n_vocab, dtype=tf.int32)
        return tf.compat.v1.where(mask, noise, context)
    else:
        return context


def main():
    args = Args(
                dataset="../data/sciclone/data10/mtufano/deepLearningMutants/out/changes/code",
                model_name="117M",
                combine=50000,
                batch_size=1,
                learning_rate=0.00002,
                optimizer="sgd",
                noise=0.0,
                top_k=40,
                top_p=0.0,
                run_name="run1",
                sample_every=100,
                sample_length=1023,
                sample_num=1,
                save_every=1000,
                val_dataset=None,
                val_batch_size=2,
                val_batch_count=40,
                val_every=100
    )
    
    enc = get_encoder(args.model_name, "models")
    hparams = default_hparams()
#     with open(os.path.join('models', args.model_name, 'hparams.json')) as f:
#         hparams.override_from_dict(json.load(f))

    if args.sample_length > hparams.n_ctx:
        raise ValueError(
            "Can't get samples longer than window size: %s" % hparams.n_ctx)

#     if args.model_name == '345M':
#         args.memory_saving_gradients = True
#         if args.optimizer == 'adam':
#             args.only_train_transformer_layers = True

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    config.graph_options.rewrite_options.layout_optimizer = rewriter_config_pb2.RewriterConfig.OFF
    with tf.compat.v1.Session(config=config) as sess:
        context = tf.compat.v1.placeholder(tf.int32, [args.batch_size, None])
        context_in = context # randomize(context, hparams, args.noise)
        output = model(hparams=hparams, X=context_in)
        loss = tf.reduce_mean(
            input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=context[:, 1:], logits=output['logits'][:, :-1]))

        if args.val_every > 0:
            val_context = tf.compat.v1.placeholder(tf.int32, [args.val_batch_size, None])
            val_output = model(hparams=hparams, X=val_context)
            val_loss = tf.reduce_mean(
                input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=val_context[:, 1:], logits=val_output['logits'][:, :-1]))
            val_loss_summary = tf.compat.v1.summary.scalar('val_loss', val_loss)


        tf_sample = sample_sequence(
            hparams=hparams,
            length=args.sample_length,
            context=context,
            batch_size=args.batch_size,
            temperature=1.0,
            top_k=args.top_k)

        all_vars = [v for v in tf.compat.v1.trainable_variables() if 'model' in v.name]
        train_vars = all_vars # [v for v in all_vars if '/h' in v.name] if args.only_train_transformer_layers else all_vars

        if args.optimizer == 'adam':
            opt = tf.compat.v1.train.AdamOptimizer(learning_rate=args.learning_rate)
        elif args.optimizer == 'sgd':
            opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=args.learning_rate)
        else:
            exit('Bad optimizer:', args.optimizer)

        opt_grads = tf.gradients(ys=loss, xs=train_vars)
        opt_grads = list(zip(opt_grads, train_vars))
        opt_apply = opt.apply_gradients(opt_grads)
        summary_loss = tf.compat.v1.summary.scalar('loss', loss)

        summary_lr = tf.compat.v1.summary.scalar('learning_rate', args.learning_rate)
        summaries = tf.compat.v1.summary.merge([summary_lr, summary_loss])

        summary_log = tf.compat.v1.summary.FileWriter(
            os.path.join(CHECKPOINT_DIR, args.run_name))

        saver = tf.compat.v1.train.Saver(
            var_list=all_vars,
            max_to_keep=5,
            keep_checkpoint_every_n_hours=2)
        sess.run(tf.compat.v1.global_variables_initializer())

        ckpt = tf.train.latest_checkpoint(
            os.path.join(CHECKPOINT_DIR, args.run_name))
        if ckpt is None:
            # Get fresh GPT weights if new run.
            ckpt = tf.train.latest_checkpoint(
                os.path.join('models', args.model_name))

        print('Loading checkpoint', ckpt)
        saver.restore(sess, ckpt)

        print('Loading dataset...')
#         chunks = load_dataset(enc, args.dataset, args.combine)
        data_sampler = Sampler(trn_set)
        if args.val_every > 0:
            val_chunks = val_set # load_dataset(enc, args.val_dataset, args.combine) if args.val_dataset else chunks
        print('dataset has', data_sampler.total_size, 'tokens')
        print('Training...')

        if args.val_every > 0:
            # Sample from validation set once with fixed seed to make
            # it deterministic during training as well as across runs.
            val_data_sampler = Sampler(val_chunks, seed=1)
            val_batches = [[val_data_sampler.sample(128) for _ in range(args.val_batch_size)]
                           for _ in range(args.val_batch_count)]

        counter = 1
        counter_path = os.path.join(CHECKPOINT_DIR, args.run_name, 'counter')
        if os.path.exists(counter_path):
            # Load the step number if we're resuming a run
            # Add 1 so we don't immediately try to save again
            with open(counter_path, 'r') as fp:
                counter = int(fp.read()) + 1

        def save():
            maketree(os.path.join(CHECKPOINT_DIR, args.run_name))
            print(
                'Saving',
                os.path.join(CHECKPOINT_DIR, args.run_name,
                             'model-{}').format(counter))
            saver.save(
                sess,
                os.path.join(CHECKPOINT_DIR, args.run_name, 'model'),
                global_step=counter)
            with open(counter_path, 'w') as fp:
                fp.write(str(counter) + '\n')

        def generate_samples():
            print('Generating samples...')
            context_tokens = data_sampler.sample(1)
            all_text = []
            index = 0
            while index < args.sample_num:
                out = sess.run(
                    tf_sample,
                    feed_dict={context: args.batch_size * [context_tokens]})
                for i in range(min(args.sample_num - index, args.batch_size)):
                    text = enc.decode(out[i])
                    text = '======== SAMPLE {} ========\n{}\n'.format(
                        index + 1, text)
                    all_text.append(text)
                    index += 1
            print(text)
            maketree(os.path.join(SAMPLE_DIR, args.run_name))
            with open(
                    os.path.join(SAMPLE_DIR, args.run_name,
                                 'samples-{}').format(counter), 'w') as fp:
                fp.write('\n'.join(all_text))

        def validation():
            print('Calculating validation loss...')
            losses = []
            for batch in tqdm.tqdm(val_batches):
                losses.append(sess.run(val_loss, feed_dict={val_context: batch}))
            v_val_loss = np.mean(losses)
            v_summary = sess.run(val_loss_summary, feed_dict={val_loss: v_val_loss})
            summary_log.add_summary(v_summary, counter)
            summary_log.flush()
            print(
                '[{counter} | {time:2.2f}] validation loss = {loss:2.2f}'
                .format(
                    counter=counter,
                    time=time.time() - start_time,
                    loss=v_val_loss))

        def sample_batch():
            return [data_sampler.sample(128) for _ in range(args.batch_size)]


        avg_loss = (0.0, 0.0)
        start_time = time.time()

        try:
            while True:
                if counter % args.save_every == 0:
                    save()
                if counter % args.sample_every == 0:
                    generate_samples()
                if args.val_every > 0 and (counter % args.val_every == 0 or counter == 1):
                    validation()

#                 if args.accumulate_gradients > 1:
#                     sess.run(opt_reset)
#                     for _ in range(args.accumulate_gradients):
#                         sess.run(
#                             opt_compute, feed_dict={context: sample_batch()})
#                     (v_loss, v_summary) = sess.run((opt_apply, summaries))
#                 else:
                (_, v_loss, v_summary) = sess.run(
                    (opt_apply, loss, summaries),
                    feed_dict={context: sample_batch()})

                summary_log.add_summary(v_summary, counter)

                avg_loss = (avg_loss[0] * 0.99 + v_loss,
                            avg_loss[1] * 0.99 + 1.0)

                print(
                    '[{counter} | {time:2.2f}] loss={loss:2.2f} avg={avg:2.2f}'
                    .format(
                        counter=counter,
                        time=time.time() - start_time,
                        loss=v_loss,
                        avg=avg_loss[0] / avg_loss[1]))

                counter += 1
        except KeyboardInterrupt:
            print('interrupted')
            save()


if __name__ == '__main__':
    main()

W0703 20:37:51.112021 140155313919808 deprecation.py:323] From <ipython-input-11-ef3b52cbd52e>:12: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0703 20:38:00.931291 140155313919808 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading checkpoint models/117M/model.ckpt



  0%|          | 0/40 [00:00<?, ?it/s]

Loading dataset...
dataset has 15705181 tokens
Training...
Calculating validation loss...



100%|██████████| 40/40 [00:02<00:00, 11.05it/s]

...

[700 | 147.92] validation loss = 2.03
[700 | 147.98] loss=2.26 avg=1.92
[701 | 148.04] loss=0.40 avg=1.91
[702 | 148.09] loss=1.58 avg=1.91
[703 | 148.15] loss=2.26 avg=1.91
[704 | 148.20] loss=2.63 avg=1.92
[705 | 148.25] loss=2.55 avg=1.92
[706 | 148.31] loss=2.51 avg=1.93
[707 | 148.36] loss=1.53 avg=1.93
[708 | 148.41] loss=2.39 avg=1.93
[709 | 148.47] loss=1.07 avg=1.92
[710 | 148.52] loss=1.48 avg=1.92
[711 | 148.57] loss=2.13 avg=1.92
[712 | 148.63] loss=3.54 avg=1.94
[713 | 148.68] loss=2.58 avg=1.94
[714 | 148.74] loss=2.73 avg=1.95
[715 | 148.79] loss=0.69 avg=1.94
[716 | 148.84] loss=1.77 avg=1.94
[717 | 148.90] loss=1.62 avg=1.93
[718 | 148.95] loss=1.12 avg=1.92
[719 | 149.01] loss=1.36 avg=1.92
[720 | 149.06] loss=1.83 avg=1.92
[721 | 149.11] loss=1.68 avg=1.91
[722 | 149.17] loss=2.68 avg=1.92
[723 | 149.23] loss=2.31 avg=1.93
[724 | 149.28] loss=0.63 avg=1.91
[725 | 149.34] loss=3.22 avg=1.93
[726 | 149.39] loss=1.62 avg=1.92
[727 | 149.44] loss=2.01 avg=1.92
[728 | 149


 12%|█▎        | 5/40 [00:00<00:00, 45.23it/s]

======== SAMPLE 1 ========
 new by a local entrepreneur named Eric. He was looking for the right thing to do with the time he had left when he left to pursue his passion for video games for a living. Eric was a huge fan and soon decided to share his ideas with him. He brought this dream to life and has been working on this project all year to date. He is now working on a way for the game to work and will continue to work on it after its release.

The team at Nintendo have made a fantastic debut on Kickstarter. The video game, Wii U was funded with only 40 backers (just 3 of this year). This is a project that has seen the development of new technology like a game or a game controller. In addition, the team have also created a Kickstarter campaign that will allow the crowd in this game to be helped through its final release. The amount of money they are getting into this is a testament to how generous of a team their passion is. It will take an amazing amount of hard work from the entire


100%|██████████| 40/40 [00:00<00:00, 41.83it/s]

[800 | 167.84] validation loss = 2.02
[800 | 167.90] loss=2.52 avg=1.91
[801 | 167.96] loss=2.10 avg=1.91
[802 | 168.01] loss=2.39 avg=1.92
[803 | 168.07] loss=2.82 avg=1.93
[804 | 168.12] loss=2.88 avg=1.94
[805 | 168.17] loss=2.06 avg=1.94
[806 | 168.23] loss=1.47 avg=1.93
[807 | 168.28] loss=2.08 avg=1.94
[808 | 168.34] loss=1.59 avg=1.93
[809 | 168.39] loss=1.83 avg=1.93
[810 | 168.44] loss=1.05 avg=1.92
[811 | 168.50] loss=1.19 avg=1.91
[812 | 168.55] loss=2.48 avg=1.92
[813 | 168.60] loss=1.05 avg=1.91
[814 | 168.66] loss=1.98 avg=1.91
[815 | 168.71] loss=2.46 avg=1.92
[816 | 168.77] loss=1.83 avg=1.92
[817 | 168.82] loss=1.82 avg=1.92
[818 | 168.87] loss=2.00 avg=1.92
[819 | 168.92] loss=1.15 avg=1.91
[820 | 168.98] loss=2.79 avg=1.92
[821 | 169.04] loss=3.30 avg=1.93
[822 | 169.09] loss=1.87 avg=1.93
[823 | 169.14] loss=1.62 avg=1.93
[824 | 169.20] loss=1.89 avg=1.93
[825 | 169.26] loss=1.90 avg=1.93
[826 | 169.31] loss=1.82 avg=1.93
[827 | 169.36] loss=2.39 avg=1.93
[828 | 169

In [23]:
# Load the tensorboard notebook extension
%load_ext tensorboard

In [24]:
%tensorboard --logdir checkpoint/run1/